In [1]:
import os
os.chdir('..')

os.environ["DJANGO_SETTINGS_MODULE"] = "course_management.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ["IS_LOCAL"] = "1"

import django
django.setup()

IS_LOCAL=True
['C:\\Users\\alexe\\git\\course-management-platform\\.venv\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\alexe\\AppData\\Roaming\\jupyter\\runtime\\kernel-57c2d1c8-fa76-4514-a88e-b4c9a31f5314.json']
Is test: False


In [2]:
from courses.models import *

In [12]:
from django.db.models import Prefetch

In [4]:
course = Course.objects.get(id=4)
course

<Course: LLM Zoomcamp 2024>

In [6]:
homework = Homework.objects.get(course=course, slug='competition')
homework

<Homework: LLM Zoomcamp 2024 - LLM Zoomcamp 2024 Competition>

In [16]:
answers_prefetch = Prefetch("answer_set", queryset=Answer.objects.all())

In [17]:
submissions = Submission.objects.prefetch_related(answers_prefetch).filter(homework=homework)

In [37]:
for s in submissions:
    print(s.id) #, s.student)
    
    answers = list(s.answer_set.values("id", "question_id", "answer_text"))
    print(answers)

11533
[{'id': 63299, 'question_id': 153, 'answer_text': 'Valery Rabinin'}, {'id': 63300, 'question_id': 154, 'answer_text': '0.85'}]
11543
[{'id': 63355, 'question_id': 153, 'answer_text': 'Marcel Castro'}, {'id': 63356, 'question_id': 154, 'answer_text': '0.9'}]
11340
[{'id': 62217, 'question_id': 153, 'answer_text': 'Alexander Heinz'}, {'id': 62218, 'question_id': 154, 'answer_text': '0.85000'}]
11341
[{'id': 62219, 'question_id': 153, 'answer_text': 'Vyacheslav Shen'}, {'id': 62220, 'question_id': 154, 'answer_text': '0.95000'}]
11342
[{'id': 62221, 'question_id': 153, 'answer_text': 'CHINONSO ODIAKA'}, {'id': 62222, 'question_id': 154, 'answer_text': '0.9'}]
11343
[{'id': 62223, 'question_id': 153, 'answer_text': 'vladkha'}, {'id': 62224, 'question_id': 154, 'answer_text': '0.95000'}]
11344
[{'id': 62225, 'question_id': 153, 'answer_text': 'Fazle Hasan'}, {'id': 62226, 'question_id': 154, 'answer_text': '0.55000'}]
11345
[{'id': 62227, 'question_id': 153, 'answer_text': 'Dimitrios 

## Custom logic

Updates

In [27]:
a = Answer.objects.get(id=62222)
a.answer_text = '0.9'
a.save()

Exclude

In [ ]:
score_max = 0.95
score_min = 0.50

points_max = 20
points_min = 10

In [31]:
ignore = {11346}

In [32]:
def calculate_points(score):
    score_max = 0.95
    score_min = 0.50
    points_max = 20
    points_min = 10

    if score >= score_max:
        return points_max
    elif score < score_min:
        return 0
    elif score == score_min:
        return points_min
   
    score_ratio = (score - score_min) / (score_max - score_min)
    points = points_min + score_ratio * (points_max - points_min)
    return round(points)

In [36]:
calculate_points(0.8)

17

In [52]:
for s in submissions:
    if s.id in ignore:
        continue

    print(s.id)
    
    answers_raw = list(s.answer_set.all())
    answers = {d.question_id: d for d in answers_raw}

    answer_name = answers[153]
    answer_name.correct = True
    answer_name.save()
    
    answer_score = answers[154]
    answer_score.correct = True
    answer_score.save()

    score = float(answer_score.answer_text)
    points = calculate_points(score)

    s.questions_score = points
    s.total_score = points
    s.save()
    print(score, points)

11533
0.85 18
11543
0.9 19
11340
0.85 18
11341
0.95 20
11342
90.0 20
11343
0.95 20
11344
0.55 11
11345
0.85 18
11350
0.9 19
11356
0.95 20
11360
0.85 18
11456
0.9 19
11363
0.9 19
11367
0.9 19
11368
0.65 13
11581
0.85 18
11405
0.55 11
11586
0.8 17
11416
0.9 19
11593
0.85 18
11862
0.85 18
11429
0.85 18
11605
0.65 13
11436
0.85 18
11437
0.8 17


In [53]:
homework.state = HomeworkState.SCORED.value
homework.save()

In [54]:
from courses.scoring import update_leaderboard

In [55]:
update_leaderboard(course)

INFO Updating leaderboard for course 4
INFO Updated leaderboard in 1.59 seconds
